## Natural Language Procesing 
### Example from a pdf


Note (possible consideration for projects) -you do not need to work with a pdf for nlp!    
e.g. work with a .txt file, read off of a web page etc. 

#### Notes for HW2   

You need to make your code abstract enough to work for future docs. So if I wanted to run this code for 2017 I should not have to change more than one line of code. You don't need to verify that you get the correct answers with other years, just generalize your code enough so that it runs for other years 
     
Good code orients users to what they'll need to change up at the top

You need to use functions in this assignment. (and going forward)

It is ok to hard code the page numbers      
Best practice:     
- If you need to hard code, make it an argument

You'll want to try to catch negations (e.g. "will not increase", "failed to rise") 

Tip:
Writing fns: 
- Think of what it should do, and mock it out
- Write example input
- Try again with different input
- Use print statements to tell you what's happening
- Remove or comment out the intermediary output when finished

#Spacy documentation
#https://spacy.io/api/doc

In [ ]:
# import statements belong at the top of your code
import os
import requests
import PyPDF2 
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm") #English

In [ ]:
path = os.getcwd()

In [ ]:
url = 'https://countyofsb.org/ceo/asset.c/4171'
filename = 'FY_2020_21_Section_B_Executive_Summary.pdf'

In [ ]:
# make a comment for where this function is called
# e.g. called in main()
# but for this example, I'm calling my fns imediately (to demonstrate)
def get_pdf(url, filename, path):
    response = requests.get(url)
    with open(os.path.join(path, filename), 'wb') as ofile:
        ofile.write(response.content)



if filename not in os.listdir():
    print('downloading document from {}'.format(url))
    get_pdf(url, filename, path)
else:
    print('document already in {}'.format(path))

In [ ]:
os.listdir()

In [ ]:
def read_pdf(filename, path):
    with open(os.path.join(path, filename), 'rb') as ifile:
        pdf = PyPDF2.PdfFileReader(ifile)

        print('Number of pages:', pdf.numPages)

        pages = []
        for p in range(pdf.numPages):
            page = pdf.getPage(p)
            text = page.extractText()
            text = text.replace("™", "'")
            text = text.replace("\n", "")
            pages.append(text)
        
        return pages

pages = read_pdf(filename, path)

In [ ]:
pages[0]

In [ ]:
pages[2]

In [ ]:
pages[2][0:9]

In [ ]:
def tokenize(pages, page_num):
    text = pages[page_num]
    doc = nlp(text)
    return doc

tokenized_page = tokenize(pages, 4)

In [ ]:
tokenized_page[0]

In [ ]:
list(tokenized_page[50].ancestors)

In [ ]:
list(tokenized_page[50].children)

#### Exploring our page

In [ ]:
covid_terms = ['pandemic', 'COVID']
covid_tokens = [t for t in tokenized_page if any([e in t.string for e in covid_terms])]
covid_tokens

In [ ]:
covid_ancs = [list(t.ancestors) for t in covid_tokens]
covid_ancs

In [ ]:
# nested for loop
for ancs in covid_ancs:
    for anc in ancs:
        print(anc, anc.pos_)

In [ ]:
covid_anc_type = [[(anc, anc.pos_) for anc in ancs] for ancs in covid_ancs]
covid_anc_type

In [ ]:
covid_ancs_verbs = [[a for a in ancs if a.pos_ == 'VERB'] for ancs in covid_ancs]
covid_ancs_verbs

In [ ]:
for token_list in econ_ancestors:
    for ancestor in token_list:
        print(ancestor.dep_)

In [ ]:
covid_children = [list(t.children) for t in covid_tokens]
covid_children

In [ ]:
children_of_covid_ancs = [[list(a.children) for a in ancs] for ancs in covid_ancs]
children_of_covid_ancs

In [ ]:
list(tokenized_page[0:10].noun_chunks)

In [ ]:
list(tokenized_page[10:100].noun_chunks)

In [ ]:
test = list(tokenized_page[45:47].noun_chunks)
test

In [ ]:
test[0].root

In [ ]:
budget_nchunks = [nc for nc in tokenized_page.noun_chunks if 'budget' in nc.string]
budget_nchunks

In [ ]:
covid_ancs

In [ ]:
for t_list in covid_ancs:
    #print(t_list) #debug
    for token in t_list:
        #print(token) #debug
        if str(token) == 'accelerated':
            print('ancestor', list(token.ancestors))
            print('child', list(token.children))

In [ ]:
for t_list in covid_ancs:
    for token in t_list:
        if str(token) == 'accelerated':
            accelerated_anc = list(token.ancestors) # expect "is"

for token in accelerated_anc:
    print(list(token.ancestors))
    print(list(token.children))

**This can all be deeply unsatisfying, and you're going to hit a lot of dead ends. Sometimes you do just have to use a brute-force approach though. 
Try enough things and you'll get what you're looking for**

### Let's take look at just one sentence

In [ ]:
text = 'The COVID19 pandemic has caused a national recession'
doc = nlp(text)
doc

In [ ]:
for token in doc:
    print(token, list(token.ancestors))

In [ ]:
for token in doc:
    print(token, list(token.children))

In [ ]:
img = spacy.displacy.render(doc, style='dep', options={'distance' : 140}, jupyter=False)
with open('dependency parser.svg', 'w', encoding='utf-8') as f:
    f.write(img)

In [ ]:
# subtree pulls the token, its ancestors and its children
for token in doc:
    print(token, list(token.subtree))

In [ ]:
nc = list(doc.noun_chunks)
nc

In [ ]:
nc[0].root

In [ ]:
nc[1].root